# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from GridCal.Engine import *
from GridCal.Engine.IO.file_handler import FileOpen
from GridCal.Engine.Devices.shunt import Shunt
from GridCal.Engine.Simulations.PowerFlow.time_series_driver import TimeSeries
from GridCal.Engine.Simulations.PowerFlow.power_flow_driver import PowerFlowDriver

Bentayga is not available
Newton native is not available
Alliander's power grid model is not available, try pip install power-grid-model
Newton Power Analytics is not available: No module named 'newtonpa'


## Loading the PSS/E Model

First, we load a PSS/E model from a `*.raw` file. We construct a variable that points to the absolute path of the PSS/E file. Then, we use the `FileOpen` class to load it into a `file_handler` instance. Then, we instantiate a grid object and assign it to the results of parsing the PSS/E file into a GridCal model.

In [6]:
path_raw = os.path.abspath(os.path.join("PSSE_Files", "TwoAreas_Base_Case.raw"))

file_handler = FileOpen(path_raw)

# Initializing a grid object
grid = None

# Opening the grid object
grid = file_handler.open()

grid.name = "Kundur Two Areas"

## Visualizing the Network

We use the GridCal functionalities to visualize the topology of the network as a graph. This is done by invoking the `plot_graph()` function in the grid object.

In [ ]:
grid.plot_graph()
plt.show()

## Setting Up Power Flow Solver

The options to the power flow driver object are passed in a `PowerFlowOptions` object that contains:
- the solver type;
- the option to start from an existing solution;
- a flag to indicate if the power flow computation shall be parallelized or not;
- the maximum number of iterations;
- the reactive power control mode (in case the $Q$ limits are hit for PV buses).

Once the options are specified, a `PowerFlowDriver` object can be instantiated passing the grid model and the options as arguments.

In [7]:
options = PowerFlowOptions(SolverType.NR, 
                          verbose = True,
                          initialize_with_existing_solution = False,
                          multi_core = False,
                          tolerance = 1e-6,
                          max_iter = 99,
                          control_q = ReactivePowerControlMode.Direct)

pf = PowerFlowDriver(grid, options)

## Running a Power Flow

A power flow computation for a single snapshot is run using the function `run()` of the `PowerFlowDriver` class.

In [9]:
pf.run()

## Printing and Exporting Results

The results of the power flow computation are stored in the field `results` inside the `PowerFlowDriver` object.

In [10]:
print('\n\n', grid.name)
print('\t|V|:', abs(pf.results.voltage))
print('\t|Sbranch|:', abs(pf.results.Sf))
print('\t|loading|:', abs(pf.results.loading) * 100)
print('\terr:', pf.results.error)
print('\tConv:', pf.results.converged)



 Kundur Two Areas
	|V|: [1.03       1.01       1.03       1.01       1.00644977 0.97812285 0.96100702 0.94859949 0.9713535  0.98345116 1.0082484 ]
	|Sbranch|: [ 724.04272354  738.27384039  740.329235    728.59464792  707.4879902  1393.60667541  200.25933915  200.25933915  196.87793307  196.87793307 1391.112828    707.03601625]
	|loading|: [7.24042724e+13 7.38273840e+13 7.40329235e+13 7.28594648e+13 7.07487990e+13 1.39360668e+14 2.00259339e+13 2.00259339e+13 1.96877933e+13 1.96877933e+13 1.39111283e+14 7.07036016e+13]
	err: 2.7658498780169793e-07
	Conv: True


It is also possible to export the data as a `*.csv` file using Pandas.

In [11]:
# Create Headers
headers = ['Vm (p.u.)', 'Va (deg)', 'Vre', 'Vim']

# Choose variables to display
Vm = np.abs(pf.results.voltage)
Va = np.angle(pf.results.voltage, deg=True)
Vre = pf.results.voltage.real
Vim = pf.results.voltage.imag
data = np.c_[Vm, Va, Vre, Vim]

# Create Data Frame
v_df = pd.DataFrame(data=data, columns=headers, index=grid.bus_names)
# Export Results
v_df.to_csv("example_pf-results.csv")